In [1]:
import ollama

In [ ]:
import sqlite3
from datetime import datetime
import pandas as pd

class Database:
    """SQLite database for storing and retrieving farming and market data."""
    
    def __init__(self, db_path="farming_system.db"):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.initialize_database()
        
    def initialize_database(self):
        """Create necessary tables if they don't exist."""
        # Farmer advisor data table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS farm_data (
            Farm_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Soil_pH REAL,
            Soil_Moisture REAL,
            Temperature_C REAL,
            Rainfall_mm REAL,
            Crop_Type TEXT,
            Fertilizer_Usage_kg REAL,
            Pesticide_Usage_kg REAL,
            Crop_Yield_ton REAL,
            Sustainability_Score REAL
        )
        ''')
        
        # Market research data table
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS market_data (
            Market_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            Product TEXT,
            Market_Price_per_ton REAL,
            Demand_Index REAL,
            Supply_Index REAL,
            Competitor_Price_per_ton REAL,
            Economic_Indicator REAL,
            Weather_Impact_Score REAL,
            Seasonal_Factor REAL,
            Consumer_Trend_Index REAL
        )
        ''')
        
        # Recommendations table for storing agent outputs
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS recommendations (
            recommendation_id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            farm_id TEXT,
            recommendation_type TEXT,
            recommendation_text TEXT,
            projected_sustainability_impact REAL,
            projected_profit_impact REAL,
            confidence_score REAL
        )
        ''')
        
        self.conn.commit()

    def import_from_csv(self, csv_path, table_type):
        try:
            # Read CSV file
            import pandas as pd
            df = pd.read_csv(csv_path)
            
            if table_type == 'farm':
                # Check required columns for farm data
                required_cols = ['Farm_ID', 'Soil_pH', 'Soil_Moisture', 'Temperature_C', 
                                'Rainfall_mm', 'Crop_Type', 'Fertilizer_Usage_kg', 
                                'Pesticide_Usage_kg', 'Crop_Yield_ton', 'Sustainability_Score']
                
                # Verify all required columns are present
                if not all(col in df.columns for col in required_cols):
                    missing = [col for col in required_cols if col not in df.columns]
                    return f"Error: Missing required columns: {missing}"
                
                # Insert records
                for _, row in df.iterrows():
                    
                    self.insert_farm_data({
                        'Farm_ID': row['Farm_ID'],
                        'Soil_pH': row['Soil_pH'],
                        'Soil_Moisture': row['Soil_Moisture'],
                        'Temperature_C': row['Temperature_C'],
                        'Rainfall_mm': row['Rainfall_mm'],
                        'Crop_Type': row['Crop_Type'],
                        'Fertilizer_Usage_kg': row['Fertilizer_Usage_kg'],
                        'Pesticide_Usage_kg': row['Pesticide_Usage_kg'],
                        'Crop_Yield_ton': row['Crop_Yield_ton'],
                        'Sustainability_Score': row['Sustainability_Score']
                    })
                
                    
            elif table_type == 'market':
                # Check required columns for market data
                required_cols = ['Market_ID', 'Product', 'Market_Price_per_ton', 
                                'Demand_Index', 'Supply_Index', 'Competitor_Price_per_ton',
                                'Economic_Indicator', 'Weather_Impact_Score', 
                                'Seasonal_Factor', 'Consumer_Trend_Index']
                
                # Verify all required columns are present
                if not all(col in df.columns for col in required_cols):
                    missing = [col for col in required_cols if col not in df.columns]
                    return f"Error: Missing required columns: {missing}"
                    
                # Insert records
                for _, row in df.iterrows():
                    self.insert_market_data({
                        'Market_ID': row['Market_ID'],
                        'Product': row['Product'],
                        'Market_Price_per_ton': row['Market_Price_per_ton'],
                        'Demand_Index': row['Demand_Index'],
                        'Supply_Index': row['Supply_Index'],
                        'Competitor_Price_per_ton': row['Competitor_Price_per_ton'],
                        'Economic_Indicator': row['Economic_Indicator'],
                        'Weather_Impact_Score': row['Weather_Impact_Score'],
                        'Seasonal_Factor': row['Seasonal_Factor'],
                        'Consumer_Trend_Index': row['Consumer_Trend_Index']
                    })
            else:
                return "Error: table_type must be either 'farm' or 'market'"
                
            return f"Successfully imported {len(df)} records into {table_type} table"
            
        except Exception as e:
            return f"Error importing data: {str(e)}"
        
    def insert_farm_data(self, data):
        """Insert new farm data into the database."""
        now = datetime.now().isoformat()
        query = '''
        INSERT INTO farm_data (
             Farm_ID, Soil_pH, Soil_Moisture, Temperature_C, 
            Rainfall_mm, Crop_Type, Fertilizer_Usage_kg, Pesticide_Usage_kg, 
            Crop_Yield_ton, Sustainability_Score
        ) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        '''
        self.cursor.execute(query, (
            data['Farm_ID'], data['Soil_pH'], data['Soil_Moisture'], 
            data['Temperature_C'], data['Rainfall_mm'], data['Crop_Type'], 
            data['Fertilizer_Usage_kg'], data['Pesticide_Usage_kg'], 
            data['Crop_Yield_ton'], data['Sustainability_Score']
        ))
        self.conn.commit()
        
    def insert_market_data(self, data):
        """Insert new market data into the database."""
        now = datetime.now().isoformat()
        query = '''
        INSERT INTO market_data (
            Market_ID, Product, Market_Price_per_ton, Demand_Index, 
            Supply_Index, Competitor_Price_per_ton, Economic_Indicator,
            Weather_Impact_Score, Seasonal_Factor, Consumer_Trend_Index
        ) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        '''
        self.cursor.execute(query, (
            data['Market_ID'], data['Product'], data['Market_Price_per_ton'], 
            data['Demand_Index'], data['Supply_Index'], data['Competitor_Price_per_ton'], 
            data['Economic_Indicator'], data['Weather_Impact_Score'], 
            data['Seasonal_Factor'], data['Consumer_Trend_Index']
        ))
        self.conn.commit()
        
    def insert_recommendation(self, data):
        """Store a new recommendation in the database."""
        now = datetime.now().isoformat()
        query = '''
        INSERT INTO recommendations (
            timestamp, Farm_ID, recommendation_type, recommendation_text,
            projected_sustainability_impact, projected_profit_impact, confidence_score
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
        '''
        self.cursor.execute(query, (
            now, data['Farm_ID'], data['recommendation_type'], data['recommendation_text'],
            data['projected_sustainability_impact'], data['projected_profit_impact'], 
            data['confidence_score']
        ))
        self.conn.commit()
        
    def get_farm_history(self, farm_id, limit=10):
        """Retrieve historical farm data for a specific farm."""
        query = '''
        SELECT * FROM farm_data 
        WHERE farm_id = ? 
        LIMIT ?
        '''
        self.cursor.execute(query, (farm_id, limit))
        columns = [col[0] for col in self.cursor.description]
        results = [dict(zip(columns, row)) for row in self.cursor.fetchall()]
        return results
    
    def get_market_trends(self, product, limit=10):
        """Retrieve historical market data for a specific product."""
        query = '''
        SELECT * FROM market_data 
        WHERE product = ? 
        ORDER BY timestamp DESC 
        LIMIT ?
        '''
        self.cursor.execute(query, (product, limit))
        columns = [col[0] for col in self.cursor.description]
        results = [dict(zip(columns, row)) for row in self.cursor.fetchall()]
        return results
    
    def get_all_farm_data(self):
        """Retrieve all farm data for model training."""
        query = "SELECT * FROM farm_data"
        return pd.read_sql_query(query, self.conn)
    
    def get_all_market_data(self):
        """Retrieve all market data for model training."""
        query = "SELECT * FROM market_data"
        return pd.read_sql_query(query, self.conn)

    def close(self):
        """Close the database connection."""
        self.conn.close()

In [ ]:
# db = Database()
# db.initialize_database()
# db.import_from_csv(r'_temp\farm_data.csv', 'farm')
# db.import_from_csv(r'_temp\market_researcher_dataset.csv', 'market')
# ## db.conn.close()

'Successfully imported 10000 records into market table'

In [1]:
import os
import sqlite3
import time

def delete_database(db_path="farming_system.db"):
    try:
        if os.path.exists(db_path):
            from sqlite3 import connect
            try:
                # Create dummy connection and close it properly
                dummy_conn = connect(db_path)
                dummy_conn.close()
                
                # On Windows, sometimes we need a small delay
                time.sleep(0.1)
            except:
                pass
                
            # Try a few times to delete the file
            max_attempts = 5
            for attempt in range(max_attempts):
                try:
                    os.remove(db_path)
                    return f"Database file '{db_path}' successfully deleted."
                except PermissionError as e:
                    if attempt < max_attempts - 1:
                        time.sleep(1)
                    else:
                        return f"Error: File still in use after {max_attempts} attempts. Please close all programs that might be using the database."
        else:
            return f"Database file '{db_path}' does not exist."
    except Exception as e:
        return f"Error deleting database: {str(e)}"

# delete_database()

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

class FarmerAdvisorAgent:
    """Agent that analyzes farm conditions and provides sustainability-focused advice."""
    
    def __init__(self, database):
        self.db = database
        self.crop_models = {}  # Models for different crops
        self.sustainability_model = None
        self.train_models()
        
    def train_models(self):
        """Train predictive models based on historical farm data."""
        data = self.db.get_all_farm_data()
        if len(data) < 10:  # Not enough data for reliable training
            return
            
        # Training models for crop-specific yield prediction
        for crop in data['Crop_Type'].unique():
            crop_data = data[data['Crop_Type'] == crop]
            if len(crop_data) >= 5:  # Minimum samples for training
                X = crop_data[['Soil_pH', 'Soil_Moisture', 'Temperature_C', 
                              'Rainfall_mm', 'Fertilizer_Usage_kg', 'Pesticide_Usage_kg']]
                y = crop_data['Crop_Yield_ton']
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                
                model = RandomForestRegressor(n_estimators=100, random_state=42)
                model.fit(X_train_scaled, y_train)
                
                self.crop_models[crop] = {
                    'model': model,
                    'scaler': scaler
                }
        
        # Train sustainability score prediction model
        X = data[['Soil_pH', 'Soil_Moisture', 'Temperature_C', 
                              'Rainfall_mm', 'Fertilizer_Usage_kg', 'Pesticide_Usage_kg', 'Crop_Yield_ton']]
        y = data['Sustainability_Score']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train_scaled, y_train)
        
        self.sustainability_model = {
            'model': model,
            'scaler': scaler
        }
    
    def predict_yield(self, farm_data, Crop_Type):
        """Predict crop yield based on farm conditions."""
        if Crop_Type not in self.crop_models:
            return None  # No model available for this crop
            
        model_info = self.crop_models[Crop_Type]
        features = np.array([[
            farm_data['Soil_pH'], 
            farm_data['Soil_Moisture'], 
            farm_data['Temperature_C'],
            farm_data['Rainfall_mm'], 
            farm_data['Fertilizer_Usage_kg'], 
            farm_data['Pesticide_Usage_kg']
        ]])
        
        features_scaled = model_info['scaler'].transform(features)
        predicted_yield = model_info['model'].predict(features_scaled)[0]
        
        return predicted_yield
    
    def predict_sustainability(self, farm_data, predicted_yield):
        """Predict sustainability score based on farm practices and yield."""
        if self.sustainability_model is None:
            return None  # Model not available
            
        features = np.array([[
            farm_data['Soil_pH'], 
            farm_data['Soil_Moisture'], 
            farm_data['Temperature_C'],
            farm_data['Rainfall_mm'], 
            farm_data['Fertilizer_Usage_kg'], 
            farm_data['Pesticide_Usage_kg'],
            predicted_yield
        ]])
        
        features_scaled = self.sustainability_model['scaler'].transform(features)
        predicted_score = self.sustainability_model['model'].predict(features_scaled)[0]
        
        return predicted_score
    
    def optimize_inputs(self, farm_data, Crop_Type):
        """Find optimal fertilizer and pesticide usage for sustainability."""
        if Crop_Type not in self.crop_models or self.sustainability_model is None:
            return farm_data['Fertilizer_Usage_kg'], farm_data['Pesticide_Usage_kg']
            
        best_sustainability = 0
        best_fertilizer = farm_data['Fertilizer_Usage_kg']
        best_pesticide = farm_data['Pesticide_Usage_kg']
        current_yield = self.predict_yield(farm_data, Crop_Type)
        
        # Grid search for optimal inputs (simplified approach)
        for fertilizer_factor in [0.7, 0.8, 0.9, 1.0]:
            for pesticide_factor in [0.7, 0.8, 0.9, 1.0]:
                test_data = farm_data.copy()
                test_data['Fertilizer_Usage_kg'] *= fertilizer_factor
                test_data['Pesticide_Usage_kg'] *= pesticide_factor
                
                predicted_yield = self.predict_yield(test_data, Crop_Type)
                
                # Only consider options that maintain at least 90% of current yield
                if predicted_yield >= 0.9 * current_yield:
                    sustainability = self.predict_sustainability(test_data, predicted_yield)
                    if sustainability > best_sustainability:
                        best_sustainability = sustainability
                        best_fertilizer = test_data['Fertilizer_Usage_kg']
                        best_pesticide = test_data['Pesticide_Usage_kg']
        
        return best_fertilizer, best_pesticide
    
    def generate_recommendations(self, farm_id):
        """Generate sustainable farming recommendations for a specific farm."""
        # Get recent farm data
        farm_history = self.db.get_farm_history(farm_id, limit=1)
        if not farm_history:
            return "Insufficient data for farm " + str(farm_id)
            
        farm_data = farm_history[0]
        Crop_Type = farm_data['Crop_Type']
        
        # Generate optimized input recommendations
        optimal_fertilizer, optimal_pesticide = self.optimize_inputs(farm_data, Crop_Type)
        
        # Calculate expected impact
        original_yield = self.predict_yield(farm_data, Crop_Type)
        original_sustainability = farm_data['Sustainability_Score']
        
        optimized_farm_data = farm_data.copy()
        optimized_farm_data['Fertilizer_Usage_kg'] = optimal_fertilizer
        optimized_farm_data['Pesticide_Usage_kg'] = optimal_pesticide
        
        new_yield = self.predict_yield(optimized_farm_data, Crop_Type)
        new_sustainability = self.predict_sustainability(optimized_farm_data, new_yield)
        
        # Generate text recommendation using LLM
        prompt = f"""
        Generate a farming recommendation focused on sustainability for a farmer with the following data:
        - Current crop: {Crop_Type}
        - Current fertilizer usage: {farm_data['Fertilizer_Usage_kg']} kg
        - Recommended fertilizer usage: {optimal_fertilizer} kg
        - Current pesticide usage: {farm_data['Pesticide_Usage_kg']} kg
        - Recommended pesticide usage: {optimal_pesticide} kg
        - Expected yield change: {(new_yield - original_yield) / original_yield * 100:.1f}%
        - Expected sustainability improvement: {(new_sustainability - original_sustainability) / original_sustainability * 100:.1f}%
        
        Focus on practical advice for implementing these changes and explain the environmental benefits.
        Keep the recommendation concise and actionable.
        """
        
        try:
            response = ollama.chat(
                model="mistral",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300
            )
            recommendation_text = response['message']['content']
        except Exception as e:
            # in case of API call fails
            recommendation_text = f"""
            Based on our analysis, we recommend:
            - Reduce fertilizer usage from {farm_data['Fertilizer_Usage_kg']} kg to {optimal_fertilizer} kg
            - Reduce pesticide usage from {farm_data['Pesticide_Usage_kg']} kg to {optimal_pesticide} kg
            These changes can maintain yield while improving sustainability.
            """
        
        recommendation = {
            'Farm_ID': farm_id,
            'recommendation_type': 'sustainability',
            'recommendation_text': recommendation_text,
            'projected_sustainability_impact': (new_sustainability - original_sustainability) / original_sustainability,
            'projected_profit_impact': (new_yield - original_yield) / original_yield,
            'confidence_score': 0.8  # Placeholder confidence score
        }
        
        # Store recommendation in database
        self.db.insert_recommendation(recommendation)
        
        return recommendation

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
farmer_Advisor = FarmerAdvisorAgent(Database()).generate_recommendations(1)
print(farmer_Advisor)

{'Farm_ID': 1, 'recommendation_type': 'sustainability', 'recommendation_text': '\n            Based on our analysis, we recommend:\n            - Reduce fertilizer usage from 131.6928438033121 kg to 118.52355942298088 kg\n            - Reduce pesticide usage from 2.958214627794372 kg to 2.3665717022354977 kg\n            These changes can maintain yield while improving sustainability.\n            ', 'projected_sustainability_impact': np.float64(0.14798524121094303), 'projected_profit_impact': np.float64(0.24674816815135242), 'confidence_score': 0.8}


In [59]:
class MarketResearcherAgent:
    """Agent that analyzes market trends and identifies profitable opportunities."""
    
    def __init__(self, database):
        self.db = database
        self.price_models = {}  # Models for different products
        self.demand_models = {}  # Models for demand prediction
        self.train_models()
        
    def train_models(self):
        """Train predictive models based on historical market data."""
        data = self.db.get_all_market_data()
        data['Seasonal_Factor'] = data['Seasonal_Factor'].map({'HIGH': 3, 'MEDIUM': 2, 'LOW': 1})
        if len(data) < 10:  # Not enough data for reliable training
            return
            
        # Train product-specific price prediction models
        for product in data['Product'].unique():
            product_data = data[data['Product'] == product]
            if len(product_data) >= 5:  # Min samples for training
                X = product_data[[ 'Supply_Index', 'Competitor_Price_per_ton',
                                 'Economic_Indicator', 'Weather_Impact_score', 'Seasonal_Factor','Consumer_Trend_Index']]
                y_price = product_data['Market_Price_per_ton']
                y_demand = product_data['Demand_Index']

                # Split the data once - now using y_price but this creates consistent X_train/X_test splits
                X_train, X_test, y_price_train, y_price_test, y_demand_train, y_demand_test = train_test_split(
                    X, y_price, y_demand, test_size=0.2, random_state=42)

                # Scale the features
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)

                # Train price model
                price_model = RandomForestRegressor(n_estimators=100, random_state=42)
                price_model.fit(X_train_scaled, y_price_train)

                # Train demand model - using the same X_train_scaled but with y_demand_train
                demand_model = RandomForestRegressor(n_estimators=100, random_state=42)
                demand_model.fit(X_train_scaled, y_demand_train)

                # Store both models
                self.price_models[product] = {
                    'model': price_model,
                    'scaler': scaler
                }

                self.demand_models[product] = {
                    'model': demand_model,
                    'scaler': scaler
                }
    
    def predict_price(self, market_data, product):
        """Predict market price based on market conditions."""
        if product not in self.price_models:
            return None  # No model available for this product
        model_info = self.price_models[product]
        features = np.array([[
            # market_data['Demand_Index'], 
            market_data['Supply_Index'], 
            market_data['Competitor_Price_per_ton'],
            market_data['Economic_Indicator'], 
            market_data['Weather_Impact_score'], 
            market_data['Seasonal_Factor'],
            market_data['Consumer_Trend_Index']
        ]])
        
        features_scaled = model_info['scaler'].transform(features)
        predicted_price = model_info['model'].predict(features_scaled)[0]
        
        return predicted_price
    
    def predict_demand(self, market_data, product):
        """Predict market demand based on market conditions."""
        if product not in self.demand_models:
            return None  # No model available for this product
            
        model_info = self.demand_models[product]
        features = np.array([[
            # market_data['Demand_Index'], 
            market_data['Supply_Index'], 
            market_data['Competitor_Price_per_ton'],
            market_data['Economic_Indicator'], 
            market_data['Weather_Impact_score'], 
            market_data['Seasonal_Factor'],
            market_data['Consumer_Trend_Index']
        ]])
        
        features_scaled = model_info['scaler'].transform(features)
        predicted_demand = model_info['model'].predict(features_scaled)[0]
        
        return predicted_demand
    
    def analyze_market_opportunities(self, market_id):
        """Identify market opportunities across different products."""
        market_data = self.db.get_all_market_data()
        market_data['Seasonal_Factor'] = market_data['Seasonal_Factor'].map({'HIGH': 3, 'MEDIUM': 2, 'LOW': 1})
        if market_data.empty:
            return "Insufficient market data available."
            
        # Get the most recent data for each product provided timestamp is avaialble
        latest_data = market_data.groupby('Product').last().reset_index()
        
        opportunities = []
        for _, row in latest_data.iterrows():
            product = row['Product']
            
            # Skip if no models available
            if product not in self.price_models or product not in self.demand_models:
                continue
                
            # Create market data dict for predictions
            market_dict = {
                # 'Demand_Index': row['Demand_Index'],
                'Supply_Index': row['Supply_Index'],
                'Competitor_Price_per_ton': row['Competitor_Price_per_ton'],
                'Economic_Indicator': row['Economic_Indicator'],
                'Weather_Impact_score': row['Weather_Impact_score'],
                'Seasonal_Factor': row['Seasonal_Factor'],
                'Consumer_Trend_Index': row['Consumer_Trend_Index']
            }
            
            # Make predictions
            price_now = row['Market_Price_per_ton']
            demand_now = row['Demand_Index']
            
            # Predict future (assuming some time horizon)
            # This is a simple approach - in reality, you'd want to forecast with time series
            predicted_price = self.predict_price(market_dict, product)
            predicted_demand = self.predict_demand(market_dict, product)
            
            # Calculate opportunity score
            price_change = (predicted_price - price_now) / price_now
            demand_change = (predicted_demand - demand_now) / demand_now
            opportunity_score = price_change + demand_change  # Simple scoring
            
            opportunities.append({
                'product': product,
                'current_price': price_now,
                'predicted_price': predicted_price,
                'price_change_pct': price_change * 100,
                'current_demand': demand_now,
                'predicted_demand': predicted_demand,
                'demand_change_pct': demand_change * 100,
                'opportunity_score': opportunity_score
            })
        
        # Sort by opportunity score
        opportunities.sort(key=lambda x: x['opportunity_score'], reverse=True)
        
        return opportunities
    
    def generate_recommendations(self, market_id):
        """Generate market recommendations based on analyzed opportunities."""
        opportunities = self.analyze_market_opportunities(market_id)
        if isinstance(opportunities, str):  # Error message
            return opportunities
        
        if not opportunities:
            return "No market opportunities identified with current data."
        
        # Get top opportunities
        top_opportunities = opportunities[:3]
        
        # Generate text recommendation using LLM
        opportunities_text = "\n".join([
            f"- {op['product']}: Price forecast change: {op['price_change_pct']:.1f}%, " +
            f"Demand forecast change: {op['demand_change_pct']:.1f}%"
            for op in top_opportunities
        ])
        
        prompt = f"""
        Generate a market analysis recommendation for farmers based on these market forecasts:
        
        {opportunities_text}
        
        Include practical advice on:
        1. Which crops show the most promise for profitability
        2. Timing considerations for planting and harvesting
        3. Market positioning and potential premium niches
        
        Keep the recommendation concise and actionable.
        """
        
        try:
            response = ollama.chat(
                model="mistral",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300
            )
            recommendation_text = response['message']['content']
        except Exception as e:
            # Fallback if API call fails
            recommendation_text = f"""
            Based on our market analysis, we recommend focusing on these top opportunities:
            {opportunities_text}
            
            These crops show stronger pricing and demand trends in the coming season.
            """
        
        recommendation = {
            'Farm_ID': 'ALL',  # Market recommendations may apply to all farms
            'recommendation_type': 'market',
            'recommendation_text': recommendation_text,
            'projected_sustainability_impact': 0.0,  # Market analysis doesn't directly address sustainability
            'projected_profit_impact': top_opportunities[0]['opportunity_score'] if top_opportunities else 0.0,
            'confidence_score': 0.7  # Placeholder confidence score
        }
        
        # Store recommendation in database
        self.db.insert_recommendation(recommendation)
        
        return recommendation

In [ ]:
market_Researcher = MarketResearcherAgent(Database()).generate_recommendations(1)
market_Researcher

TypeError: Database.get_all_market_data() missing 1 required positional argument: 'self'

In [71]:
class CoordinatorAgent:
    """Integrates recommendations from specialized agents to provide holistic advice."""
    
    def __init__(self, database, farmer_advisor, market_researcher):
        self.db = database
        self.farmer_advisor = farmer_advisor
        self.market_researcher = market_researcher
    
    def generate_integrated_recommendation(self, farm_id, market_id):
        """Generate integrated recommendations combining farm and market insights."""
        # Get specialized recommendations
        farm_recommendation = self.farmer_advisor.generate_recommendations(farm_id)
        market_recommendation = self.market_researcher.generate_recommendations(market_id)
        
        if isinstance(farm_recommendation, str) or isinstance(market_recommendation, str):
            error_message = []
            if isinstance(farm_recommendation, str):
                error_message.append(f"Farm advisor error: {farm_recommendation}")
            if isinstance(market_recommendation, str):
                error_message.append(f"Market researcher error: {market_recommendation}")
            return "\n".join(error_message)
        
        # Get recent farm data for context
        farm_history = self.db.get_farm_history(farm_id, limit=1)
        if not farm_history:
            return "Cannot generate integrated recommendation: No data for farm " + farm_id
        
        farm_data = farm_history[0]
        
        # Create prompt for integrated recommendation
        prompt = f"""
        Create an integrated farming recommendation that balances sustainability and profitability.
        
        FARM CONTEXT:
        - Farm ID: {farm_id}
        - Current crop: {farm_data['Crop_Type']}
        - Current sustainability score: {farm_data['Sustainability_Score']}
        
        SUSTAINABILITY RECOMMENDATION:
        {farm_recommendation['recommendation_text']}
        
        MARKET RECOMMENDATION:
        {market_recommendation['recommendation_text']}
        
        Create a balanced recommendation that:
        1. Prioritizes long-term sustainability while being economically viable
        2. Identifies specific actions the farmer should take
        3. Explains expected outcomes and benefits
        4. Acknowledges potential challenges and how to address them
        
        The recommendation should be practical, specific, and actionable.
        """
        
        try:
            response = ollama.chat(
                model="mistral",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300
            )
            integrated_text = response['message']['content']
            
        except Exception as e:
            # Fallback if API call fails
            integrated_text = f"""
            INTEGRATED RECOMMENDATION:
            
            SUSTAINABILITY: {farm_recommendation['recommendation_text']}
            
            MARKET CONSIDERATIONS: {market_recommendation['recommendation_text']}
            
            We recommend implementing the sustainability practices above while considering the market trends noted.
            """
        
        # Calculate weighted impact scores
        sustainability_weight = 0.6  # Prioritize sustainability
        profit_weight = 0.4
        
        integrated_sustainability_impact = (
            farm_recommendation['projected_sustainability_impact'] * sustainability_weight + 
            market_recommendation['projected_sustainability_impact'] * profit_weight
        )
        
        integrated_profit_impact = (
            farm_recommendation['projected_profit_impact'] * sustainability_weight + 
            market_recommendation['projected_profit_impact'] * profit_weight
        )
        
        # Average confidence scores
        confidence = (farm_recommendation['confidence_score'] + market_recommendation['confidence_score']) / 2
        
        recommendation = {
            'Farm_ID': farm_id,
            'recommendation_type': 'integrated',
            'recommendation_text': integrated_text,
            'projected_sustainability_impact': integrated_sustainability_impact,
            'projected_profit_impact': integrated_profit_impact,
            'confidence_score': confidence
        }
        
        # Store recommendation in database
        self.db.insert_recommendation(recommendation)
        
        return recommendation


In [73]:
cordinator = CoordinatorAgent(Database(),FarmerAdvisorAgent(Database()) ,MarketResearcherAgent(Database()))
co = cordinator.generate_integrated_recommendation(1, 1)
print(co)

{'Farm_ID': 1, 'recommendation_type': 'integrated', 'recommendation_text': '\n            INTEGRATED RECOMMENDATION:\n\n            SUSTAINABILITY: \n            Based on our analysis, we recommend:\n            - Reduce fertilizer usage from 131.6928438033121 kg to 118.52355942298088 kg\n            - Reduce pesticide usage from 2.958214627794372 kg to 2.3665717022354977 kg\n            These changes can maintain yield while improving sustainability.\n            \n\n            MARKET CONSIDERATIONS: \n            Based on our market analysis, we recommend focusing on these top opportunities:\n            - Soybean: Price forecast change: 14.6%, Demand forecast change: 22.1%\n- Corn: Price forecast change: 13.8%, Demand forecast change: 6.7%\n- Wheat: Price forecast change: -12.0%, Demand forecast change: 8.9%\n\n            These crops show stronger pricing and demand trends in the coming season.\n            \n\n            We recommend implementing the sustainability practices abo

In [75]:
class SustainableFarmingSystem:
    """Main system that coordinates the multi-agent framework."""
    
    def __init__(self):
        self.db = Database()
        self.farmer_advisor = FarmerAdvisorAgent(self.db)
        self.market_researcher = MarketResearcherAgent(self.db)
        self.coordinator = CoordinatorAgent(self.db, self.farmer_advisor, self.market_researcher)
    
    def load_sample_data(self):
        """Load sample data for demonstration purposes."""
        # Sample farm data
        farm_data = [
            {
                'Farm_ID': 10001,
                'Soil_pH': 6.5,
                'Soil_Moisture': 0.35,
                'Temperature_C': 22.5,
                'Rainfall_mm': 780,
                'Crop_Type': 'Wheat',
                'Fertilizer_Usage_kg': 120,
                'Pesticide_Usage_kg': 5.2,
                'Crop_Yield_ton': 4.8,
                'Sustainability_Score': 7.2
            },
            # {
            #     'Farm_ID': 1,
            #     'Soil_pH': 6.7,
            #     'Soil_Moisture': 0.32,
            #     'Temperature_C': 23.1,
            #     'Rainfall_mm': 750,
            #     'Crop_Type': 'Wheat',
            #     'Fertilizer_Usage_kg': 115,
            #     'Pesticide_Usage_kg': 4.8,
            #     'Crop_Yield_ton': 5.1,
            #     'Sustainability_Score': 7.5
            # },
            # {
            #     'Farm_ID': 2,
            #     'Soil_pH': 7.1,
            #     'Soil_Moisture': 0.28,
            #     'Temperature_C': 24.3,
            #     'Rainfall_mm': 620,
            #     'Crop_Type': 'Corn',
            #     'Fertilizer_Usage_kg': 150,
            #     'Pesticide_Usage_kg': 6.5,
            #     'Crop_Yield_ton': 7.8,
            #     'Sustainability_Score': 6.8
            # }
        ]
        
        # Sample market data
        market_data = [
            {
                'Market_ID': 10001,
                'Product': 'Wheat',
                'Market_Price_per_ton': 245.50,
                'Demand_Index': 8.2,
                'Supply_Index': 7.5,
                'Competitor_Price_per_ton': 240.75,
                'Economic_Indicator': 0.85,
                'Weather_Impact_Score': 0.2,
                'Seasonal_Factor': 1.1,
                'Consumer_Trend_Index': 0.95
            },
            # {
            #     'Market_ID': 1,
            #     'Product': 'Corn',
            #     'Market_Price_per_ton': 180.25,
            #     'Demand_Index': 7.8,
            #     'Supply_Index': 8.1,
            #     'Competitor_Price_per_ton': 178.50,
            #     'Economic_Indicator': 0.85,
            #     'Weather_Impact_Score': 0.15,
            #     'Seasonal_Factor': 0.9,
            #     'Consumer_Trend_Index': 0.92
            # }
        ]
        
        # Insert sample data into the database
        for farm in farm_data:
            self.db.insert_farm_data(farm)
        
        for market in market_data:
            self.db.insert_market_data(market)
    
    def generate_recommendations(self, farm_id, market_id):
        """Generate integrated recommendations for a specific farm and market."""
        return self.coordinator.generate_integrated_recommendation(farm_id, market_id)

# Instantiate the system
system = SustainableFarmingSystem()

# Load sample data
system.load_sample_data()

# Generate recommendations
recommendation = system.generate_recommendations(farm_id=10001, market_id=10001)
print(recommendation)


{'Farm_ID': 10001, 'recommendation_type': 'integrated', 'recommendation_text': '\n            INTEGRATED RECOMMENDATION:\n\n            SUSTAINABILITY: \n            Based on our analysis, we recommend:\n            - Reduce fertilizer usage from 120.0 kg to 120.0 kg\n            - Reduce pesticide usage from 5.2 kg to 4.680000000000001 kg\n            These changes can maintain yield while improving sustainability.\n            \n\n            MARKET CONSIDERATIONS: \n            Based on our market analysis, we recommend focusing on these top opportunities:\n            - Wheat: Price forecast change: 10.4%, Demand forecast change: 1439.3%\n- Soybean: Price forecast change: 14.6%, Demand forecast change: 22.1%\n- Corn: Price forecast change: 13.8%, Demand forecast change: 6.7%\n\n            These crops show stronger pricing and demand trends in the coming season.\n            \n\n            We recommend implementing the sustainability practices above while considering the market tr

{'Farm_ID': 10001, 
'recommendation_type': 'integrated', 
'recommendation_text': 
'INTEGRATED RECOMMENDATION:\n\n            SUSTAINABILITY: \n            Based on our analysis, we recommend:\n            - Reduce fertilizer usage from 120.0 kg to 120.0 kg\n            - Reduce pesticide usage from 5.2 kg to 4.680000000000001 kg\n            These changes can maintain yield while improving sustainability.\n            \n\n            MARKET CONSIDERATIONS: \n            Based on our market analysis, we recommend focusing on these top opportunities:\n            - Wheat: Price forecast change: 10.4%, Demand forecast change: 1439.3%\n- Soybean: Price forecast change: 14.6%, Demand forecast change: 22.1%\n- Corn: Price forecast change: 13.8%, Demand forecast change: 6.7%\n\n            These crops show stronger pricing and demand trends in the coming season.\n            \n\n            We recommend implementing the sustainability practices above while considering the market trends noted.\n            ', 'projected_sustainability_impact': np.float64(4.073123003923505), 'projected_profit_impact': np.float64(5.786554749182881), 'confidence_score': 0.75}

In [2]:

# Load and use a model (e.g., llama3)
response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': 'Tell me a joke!'}])

print(response['message']['content'])

 Why don't scientists trust atoms?

Because they make up everything, even jokes! (But the funny thing is, that's not really a joke... it's more of a play on words!)


In [3]:
ollama.embeddings('mistral', 'This is an AI model.')


EmbeddingsResponse(embedding=[4.017604827880859, -3.0176756381988525, -1.7140069007873535, -5.4000372886657715, 1.2987507581710815, -3.0112264156341553, -1.8625752925872803, -0.43143707513809204, 1.630493402481079, -6.172450065612793, -1.342328429222107, -4.358443737030029, -2.7880258560180664, 11.717287063598633, 1.0528284311294556, 1.2017115354537964, -8.143241882324219, -0.6286262273788452, -0.8165562748908997, -4.680652141571045, -0.938660204410553, -0.05866997689008713, 1.9073230028152466, 5.22093391418457, 1.8101015090942383, 0.7939960360527039, 1.2350927591323853, 1.4660179615020752, -13.766229629516602, -2.2008838653564453, -3.132199287414551, -5.693976879119873, -4.720168113708496, -0.042976997792720795, -2.1543972492218018, -3.4741873741149902, -1.3638911247253418, 7.399643421173096, -7.080208778381348, -0.9330751895904541, -0.8474470376968384, -3.6868133544921875, 2.3314568996429443, 1.103339433670044, -1.9930217266082764, 4.5455498695373535, -5.045601844787598, -4.958210468